In [1]:
# Identify 1 & 2 word ngrams unique to each category
# Identify 1 & 2 word ngrams unique to a subset of categories

# Try the above w/ and w/o Lemmatization

# Add extra weight to unique ngrams in the title?

In [37]:
# import module(s) into namespace
import pandas as pd
import numpy as np
import requests
import nltk

import sys
reload(sys)
sys.setdefaultencoding("utf-8")

pathname = "./Train/" #where to get/put files
pd.set_option('display.max_colwidth', 500) #important for getting all the text

In [38]:
# BIOLOGY

filename = pathname+"biology.csv"
biologydf = pd.read_csv(filename, index_col = 0) 
biologydf.shape

(13196, 3)

In [4]:
# COOKING

filename = pathname+"cooking.csv"
cookingdf = pd.read_csv(filename, index_col = 0) 
cookingdf.shape

(15404, 3)

In [5]:
# CRYPTO

filename = pathname+"crypto.csv"
cryptodf = pd.read_csv(filename, index_col = 0) 
cryptodf.shape

(10432, 3)

In [6]:
cryptodf.head()

,title,content,tags
id,,,
3,What are the benefits of the two permutation tables in DES?,"<p>Why do we use a permutation table in the first step of <a href=""http://en.wikipedia.org/wiki/Data_Encryption_Standard"" rel=""nofollow"">DES algorithm</a> and one at the end of the algorithm?</p>\n",block-cipher des permutation
7,Why use a 1-2 Oblivious Transfer instead of a 1 out of n Oblivious Transfer?,"<p>When initiating an <a href=""http://en.wikipedia.org/wiki/Oblivious_transfer"">oblivious transfer</a>, why would someone use a <a href=""http://en.wikipedia.org/wiki/Oblivious_transfer#1-2_oblivious_transfer"">1-2 oblivious transfer</a> rather than going for an <a href=""http://en.wikipedia.org/wiki/Oblivious_transfer#1-out-of-n_oblivious_transfer_and_k-out-of-n_oblivious_transfer"">1 out of n oblivious transfer</a>? Perhaps a slight time overhead for the extra message encrypts, but with everyt...",oblivious-transfer multiparty-computation function-evaluation
8,Why do we append the length of the message in SHA-1 pre-processing?,"<p>As we know, <a href=""http://en.wikipedia.org/wiki/SHA-1"">SHA-1</a> is irreversible, so why do we append the length of the message to the preimage?</p>\n",sha-1 hash
9,What is the general justification for the hardness of finding preimages for cryptographic hash functions?,"<p>Since most cryptographic hash functions are simple, compact constructions does this simplicity impose a limit on the complexity and the size of a function that can generate preimages? That is, given a cryptographic hash function, <strong>H</strong> of some length and complexity can we lower or upper bound the complexity/size of a function that finds preimages of <strong>H</strong>. If not, why not?</p>\n\n<p>If the upper bound on the size of a function that efficiently finds a preimage of...",hash cryptanalysis preimage-resistance
14,"How can I use asymmetric encryption, such as RSA, to encrypt an arbitrary length of plaintext?","<p>RSA is not designed to be used on long blocks of plaintext like a block cipher, but I need to use it to send a large (encrypted) message.</p>\n\n<p>How can I do this?</p>\n",encryption rsa public-key


In [7]:
# DIY

filename = pathname+"diy.csv"
diydf = pd.read_csv(filename, index_col = 0) 
diydf.shape

(25918, 3)

In [8]:
# ROBOTICS

filename = pathname+"robotics.csv"
roboticsdf = pd.read_csv(filename, index_col = 0) 
roboticsdf.shape

(2771, 3)

In [9]:
# TRAVEL

filename = pathname+"travel.csv"
traveldf = pd.read_csv(filename, index_col = 0) 
traveldf.shape

(19279, 3)

In [10]:
# label the categories before we combine the data frames

biologydf['category'] = u'Biology'
cookingdf['category'] = u'Cooking'
cryptodf['category'] = u'Crypto'
diydf['category'] = u'DIY'
roboticsdf['category'] = u'Robotics'
traveldf['category'] = u'Travel'

In [11]:
# Combine into a master data frame

masterdf = biologydf.append(cookingdf, ignore_index=True)
masterdf = masterdf.append(cryptodf, ignore_index=True)
masterdf = masterdf.append(diydf, ignore_index=True)
masterdf = masterdf.append(roboticsdf, ignore_index=True)
masterdf = masterdf.append(traveldf, ignore_index=True)

In [12]:
# Tokenize each title and content and reduce to a set of unique tokens in each row
from nltk import word_tokenize

masterdf['wordset'] = map(lambda x: list(set(word_tokenize(x.lower()))), masterdf['title'] + u' ' + masterdf['content'])

In [13]:
# Remove single character tokens as they aren't likely to contain value
# Remove values with '//' since it's likely just a web address

# Define a function to tell filter what we're looking for
def singleCharacter(element):
     return len(element) > 1
    
def doubleSlash(element):
     return element.find(u'//') == -1
    
# Filter our list based on that function

masterdf['wordset_filtered'] = map(lambda x: filter(singleCharacter, x), masterdf['wordset']) 
masterdf['wordset_filtered'] = map(lambda x: filter(doubleSlash, x), masterdf['wordset']) 

In [14]:
masterdf.head()

,title,content,tags,category,wordset,wordset_filtered
0,What is the criticality of the ribosome binding site relative to the start codon in prokaryotic translation?,"<p>In prokaryotic translation, how critical for efficient translation is the location of the ribosome binding site, relative to the start codon?</p>\n\n<p>Ideally, it is supposed to be -7b away from the start. How about if it is -9 bases away or even more? Will this have an observable effect on translation?</p>\n",ribosome binding-sites translation synthetic-biology,Biology,"[-7b, is, binding, site, an, relative, have, in, if, even, what, from, ideally, for, how, efficient, ribosome, ,, ., start, critical, location, more, <, ?, >, be, to, criticality, observable, codon, effect, it, translation, on, bases, about, this, of, prokaryotic, or, will, p, supposed, the, /p, -9, away]","[-7b, is, binding, site, an, relative, have, in, if, even, what, from, ideally, for, how, efficient, ribosome, ,, ., start, critical, location, more, <, ?, >, be, to, criticality, observable, codon, effect, it, translation, on, bases, about, this, of, prokaryotic, or, will, p, supposed, the, /p, -9, away]"
1,How is RNAse contamination in RNA based experiments prevented?,<p>Does anyone have any suggestions to prevent RNAse contamination when working with RNA?</p>\n\n<p>I tend to have issues with degradation regardless of whether I use DEPC treated / RNAse free water and filtered pipette tips.</p>\n,rna biochemistry,Biology,"[and, prevent, regardless, is, suggestions, tips., experiments, have, in, any, issues, use, based, working, how, when, /, tend, anyone, to, does, filtered, <, ?, >, pipette, prevented, rna, free, degradation, water, whether, contamination, rnase, with, depc, i, of, p, /p, treated]","[and, prevent, regardless, is, suggestions, tips., experiments, have, in, any, issues, use, based, working, how, when, /, tend, anyone, to, does, filtered, <, ?, >, pipette, prevented, rna, free, degradation, water, whether, contamination, rnase, with, depc, i, of, p, /p, treated]"
2,Are lymphocyte sizes clustered in two groups?,"<p>Tortora writes in <em>Principles of Anatomy and Physiology</em>:</p>\n\n<blockquote>\n <p>Lymphocytes may be as small as 6–9 μm in diameter or as large as 10–14 μm in diameter.</p>\n</blockquote>\n\n<p>Those ranges are quite close to each others. Should the above be taken to mean that lymphocytes sizes are clustered in two groups, or is it just a way of saying that lymphocytes are 6-14 μm?</p>\n",immunology cell-biology hematology,Biology,"[em, and, just, diameter, /blockquote, is, principles, ,, as, it, are, /em, in, close, should, 6-14, writes, two, ., to, above, lymphocytes, :, <, ?, /p, tortora, be, blockquote, saying, way, that, may, lymphocyte, each, quite, anatomy, taken, groups, others, clustered, μm, physiology, those, a, ranges, sizes, of, 10–14, large, p, 6–9, small, the, >, diameter., or, mean]","[em, and, just, diameter, /blockquote, is, principles, ,, as, it, are, /em, in, close, should, 6-14, writes, two, ., to, above, lymphocytes, :, <, ?, /p, tortora, be, blockquote, saying, way, that, may, lymphocyte, each, quite, anatomy, taken, groups, others, clustered, μm, physiology, those, a, ranges, sizes, of, 10–14, large, p, 6–9, small, the, >, diameter., or, mean]"
3,How long does antibiotic-dosed LB maintain good selection?,"<p>Various people in our lab will prepare a liter or so of LB, add kanamycin to 25-37 mg/L for selection, and store it at 4 °C for minipreps or other small cultures (where dosing straight LB with a 1000X stock is troublesome). Some think using it after more than a week is dubious, but we routinely use kan plates that are 1-2 months old with no ill effect.</p>\n\n<p>How long can LB with antibiotic such as kanamycin, chloramphenicol, or ampicillin be stored at 4 °C and maintain selection?</p>\n",cell-culture,Biology,"[and, troublesome, selection, routinely, mg/l, 1-2, people, effect., is, some, it, dubious, dosing, as, good, at, in, o

In [15]:
roboticsdf

,title,content,tags,category
id,,,,
1,What is the right approach to write the spin controller for a soccer robot?,<p>Imagine programming a 3 wheel soccer robot. What type of controller would you use for spinning it? P? PID?</p>\n\n<p>The goal for this controller is that it should make the robot stand in a defined angle ( 0 degree ) and turn back if rotated by hand or other robot. </p>\n\n<p>I use stepper motors for my robot and not servos so I need to implement this in my software!</p>\n\n<p>I have written a sample P type controller already and the movement is fairly good. But I would like to make it be...,soccer control,Robotics
2,How can I modify a low cost hobby servo to run 'freely'?,"<p>I've got some hobby servos (<a href=""http://www.servodatabase.com/servo/power-hd/hd-1501mg"">Power HD 1501MGs</a>) and I'd like to be able to control them (via an Arduino) so they will either go to the angle I set, or put them in a 'free running' mode, where the load will take them wherever it goes.</p>\n\n<p>Is this even possible, or am I just going to end up stripping the gears?</p>\n\n<p>My first thought is to simply kill the power to the servo, but the force required to move them in th...",control rcservo,Robotics
3,"What useful gaits exist for a six legged robot, and what are their pros and cons?","<p><a href=""http://www.oricomtech.com/projects/leg-time.htm"" rel=""nofollow"">http://www.oricomtech.com/projects/leg-time.htm</a> lists three gaits:</p>\n\n<ul>\n<li>the tripod</li>\n<li>wave, and</li>\n<li>ripple.</li>\n</ul>\n\n<p>Can these be improved, or can their relative pros and cons be altered, and are there other gaits worth considering?</p>\n",gait walk,Robotics
4,Good Microcontrollers/SOCs for a Robotics Project,"<p>I am looking for a starting point for my project, preferably using popular systems (ones there is a lot of support for). I have an Arduino Uno, a Raspberry Pi, and a lot of willpower :) Anyone here built a project using the systems above?</p>\n\n<p>Observation: I'd like to start with a simple line-following vehicle and build up afterwards.</p>\n",microcontroller arduino raspberry-pi,Robotics
5,Nearest-neighbor data structure for non-Euclidean configuration space,"<p>I'm trying to implement a nearest-neighbor structure for use in an RRT motion planner. In order to do better than a linear brute-force nearest-neighbor search, I'd like to implement something like a kd-tree. However, it seems like the classical implementation of the kd-tree assumes that each dimension of the space can be split into ""left"" and ""right"". This notion doesn't seem to apply to non-Euclidean spaces like SO(2), for instance.</p>\n\n<p>I'm working with a serial manipulator arm wit...",motion-planning rrt,Robotics
6,What good robotics software platforms / operating systems are available?,"<p>My company will soon be starting a brand new robotics project, and we are still trying to decide whether we should design and code a robotics software platform from scratch, or if there are any good existing ones.</p>\n\n<p>It would be very useful if there was a software platform which was commonly used among both academics and industry so that our robotic system was generally compatible with others, and so that people were already familiar with it.</p>\n\n<p>We would like the software pl...",software platform,Robotics
11,What software do you use to design your PCB in the robotics field?,"<p>What is the best software (despite the price) for designing the circuits and PCB boards for robots?</p>\n\n<p>I mean having lots of components, different designing methods, best accuracy, ...</p>\n\n<p>I myself use the Altium Designer which I think answers my needs, but maybe there are better ones in the market i don't know about!</p>\n",software circuit,Robotics
18,What are good methods for tuning the process noise on Kalman filters?,<p>Most often tuning the Kalman filter noise matrices is done by trial and error or domain knowledge. Are there more principled ways fo

In [16]:
# Let's see what attributes most tags have in common

#masterdf['tagset'] = map(lambda x: list(set(word_tokenize(x.lower()))), masterdf['tags'])
#biologydf['tagset'] = map(lambda x: list(set(word_tokenize(x.lower()))), biologydf['tags'])
#cookingdf['tagset'] = map(lambda x: list(set(word_tokenize(x.lower()))), cookingdf['tags'])
#cryptodf['tagset'] = map(lambda x: list(set(word_tokenize(x.lower()))), cryptodf['tags'])
#diydf['tagset'] = map(lambda x: list(set(word_tokenize(x.lower()))), diydf['tags'])
#roboticsdf['tagset'] = map(lambda x: list(set(word_tokenize(x.lower()))), roboticsdf['tags'])
#traveldf['tagset'] = map(lambda x: list(set(word_tokenize(x.lower()))), traveldf['tags'])


In [17]:
knownTags = set([])

for x in xrange(0, len(masterdf)):
    knownTags = knownTags|set(word_tokenize(masterdf.iloc[x]['tags'].lower()))
    
    
biologyTags = set([])

for x in xrange(0, len(biologydf)):
    biologyTags = biologyTags|set(word_tokenize(biologydf.iloc[x]['tags'].lower()))
    
    
cookingTags = set([])

for x in xrange(0, len(cookingdf)):
    cookingTags = cookingTags|set(word_tokenize(cookingdf.iloc[x]['tags'].lower()))
    
    
cryptoTags = set([])

for x in xrange(0, len(cryptodf)):
    cryptoTags = cryptoTags|set(word_tokenize(cryptodf.iloc[x]['tags'].lower()))
    
diyTags = set([])

for x in xrange(0, len(diydf)):
    diyTags = diyTags|set(word_tokenize(diydf.iloc[x]['tags'].lower()))
    
roboticsTags = set([])

for x in xrange(0, len(roboticsdf)):
    roboticsTags = roboticsTags|set(word_tokenize(roboticsdf.iloc[x]['tags'].lower()))
    
travelTags = set([])

for x in xrange(0, len(traveldf)):
    travelTags = travelTags|set(word_tokenize(traveldf.iloc[x]['tags'].lower()))
    
    

In [18]:
tagSeries = pd.Series(list(knownTags))
tagDF = tagSeries.to_frame(name='Tag')

tagDF['Word Count'] = map(lambda x: x.count(u'-') + 1, tagDF['Tag'])

tagDF['Biology'] = map(lambda x: 1 if x in biologyTags else 0, tagDF['Tag'])
tagDF['Cooking'] = map(lambda x: 1 if x in cookingTags else 0, tagDF['Tag'])
tagDF['Crypto'] = map(lambda x: 1 if x in cryptoTags else 0, tagDF['Tag'])
tagDF['DIY'] = map(lambda x: 1 if x in diyTags else 0, tagDF['Tag'])
tagDF['Robotics'] = map(lambda x: 1 if x in roboticsTags else 0, tagDF['Tag'])
tagDF['Travel'] = map(lambda x: 1 if x in travelTags else 0, tagDF['Tag'])

tagDF['Category_Count'] = map(lambda x: x, tagDF['Biology'] + tagDF['Cooking'] + tagDF['Crypto'] + tagDF['DIY'] + tagDF['Robotics'] + tagDF['Travel'])

In [20]:
# Subset to those included in multiple categories
overlapDF = tagDF[tagDF.Category_Count > 1]

In [21]:
overlapDF

,Tag,Word Count,Biology,Cooking,Crypto,DIY,Robotics,Travel,Category_Count
73,classification,1,1,1,0,0,0,0,2
89,glass,1,0,1,0,1,0,0,2
170,literature,1,1,0,1,0,0,1,3
221,moisture,1,0,1,0,1,0,0,2
225,database,1,1,0,1,0,0,0,2
236,information-theory,2,1,0,1,0,0,0,2
275,sugar,1,1,1,0,0,0,0,2
295,architecture,1,0,0,0,1,0,1,2
307,pets,1,1,0,0,0,0,1,2
330,hygiene,1,1,0,0,0,0,1,2


In [22]:
# Remove all instances of tags from post content
# Remove '-' characters from tags first so that we remove entire ngrams
def removeTags(element, tags):
    for x in tags:
        element = element.replace(x.replace(u'-', u' '), u'')
    return element

In [23]:
biologydf = masterdf[masterdf.category == 'Biology']
cookingdf = masterdf[masterdf.category == 'Cooking']
cryptodf = masterdf[masterdf.category == 'Crypto']
diydf = masterdf[masterdf.category == 'DIY']
roboticsdf = masterdf[masterdf.category == 'Robotics']
traveldf = masterdf[masterdf.category == 'Travel']

In [24]:
biologydf['content_no_tags'] = map(lambda x: removeTags(x, biologyTags), biologydf['content'])
cookingdf['content_no_tags'] = map(lambda x: removeTags(x, cookingTags), cookingdf['content']) 
cryptodf['content_no_tags'] = map(lambda x: removeTags(x, cryptoTags), cryptodf['content']) 
diydf['content_no_tags'] = map(lambda x: removeTags(x, diyTags), diydf['content']) 
roboticsdf['content_no_tags'] = map(lambda x: removeTags(x, roboticsTags), roboticsdf['content']) 
traveldf['content_no_tags'] = map(lambda x: removeTags(x, travelTags), traveldf['content']) 

In [25]:
biologydf['content_set'] = map(lambda x: list(set(word_tokenize(x.lower()))), biologydf['content_no_tags'])
cookingdf['content_set'] = map(lambda x: list(set(word_tokenize(x.lower()))), cookingdf['content_no_tags'])
cryptodf['content_set'] = map(lambda x: list(set(word_tokenize(x.lower()))), cryptodf['content_no_tags'])
diydf['content_set'] = map(lambda x: list(set(word_tokenize(x.lower()))), diydf['content_no_tags'])
roboticsdf['content_set'] = map(lambda x: list(set(word_tokenize(x.lower()))), roboticsdf['content_no_tags'])
traveldf['content_set'] = map(lambda x: list(set(word_tokenize(x.lower()))), traveldf['content_no_tags'])

In [26]:
biologyWords = set([])

for x in xrange(0, len(biologydf)):
    biologyWords = biologyWords|set(biologydf.iloc[x]['content_set'])
    
cookingWords = set([])

for x in xrange(0, len(cookingdf)):
    cookingyWords = cookingWords|set(cookingdf.iloc[x]['content_set'])
    
cryptoWords = set([])

for x in xrange(0, len(cryptodf)):
    cryptoWords = cryptoWords|set(cryptodf.iloc[x]['content_set'])
    
diyWords = set([])

for x in xrange(0, len(diydf)):
    diyWords = diyWords|set(diydf.iloc[x]['content_set'])
    
roboticsWords = set([])

for x in xrange(0, len(roboticsdf)):
    roboticsWords = roboticsWords|set(roboticsdf.iloc[x]['content_set'])
    
travelWords = set([])

for x in xrange(0, len(traveldf)):
    travelWords = travelWords|set(traveldf.iloc[x]['content_set'])

In [35]:
# Try to ID biology tags by removing words used in other categories and seeing what's left

biologydf['tag_guesses'] = map(lambda x: set(x).difference(cookingWords), biologydf['content_set']) 
biologydf['tag_guesses'] = map(lambda x: set(x).difference(cryptoWords), biologydf['tag_guesses'])
biologydf['tag_guesses'] = map(lambda x: set(x).difference(diyWords), biologydf['tag_guesses'])
biologydf['tag_guesses'] = map(lambda x: set(x).difference(roboticsWords), biologydf['tag_guesses'])
biologydf['tag_guesses'] = map(lambda x: set(x).difference(travelWords), biologydf['tag_guesses'])

biologydf['tag_guesses'] = map(lambda x: filter(doubleSlash, x), biologydf['tag_guesses']) 

In [36]:
biologydf[['tags', 'tag_guesses']]

,tags,tag_guesses
0,ribosome binding-sites translation synthetic-biology,"[-7b, prokaryotic]"
1,rna biochemistry,"[rnase, rna, pipette, depc]"
2,immunology cell-biology hematology,"[tortora, 6-14, 6–9, 10–14, lymocytes, μm, physiology]"
3,cell-culture,"[chloramenicol, ibiotic, 25-37, dosing, kanamycin, cultures, ampicillin]"
4,splicing mrna spliceosome introns exons,"[altetive, genomic, constitutive, mrna]"
5,dna biochemistry molecular-biology,"[sequencing, protein-bound]"
6,neuroscience synapses,"[dendritic, spines, synaptosis, axons, electrotonic]"
7,plasmids,"[parafilm, miniprep, microcentrifuge, µl, parafilmed, eppend, paper-towel]"
8,molecular-genetics gene-expression experimental-design,"[lactose, fluorescing, reporter]"
9,evolution mitochondria chloroplasts,[endosymbiont]


In [39]:
# Doesn't seem particularly accurate...
# My words seem more specific than the tags